# Cadena LCEL en funcionamiento en una aplicación RAG típica

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.ai import AIMessage
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

## Veamos cómo funciona esto con un ejemplo típico de RAG

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Carga documentos desde una URL específica, filtrando por clases HTML
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer( # Filtra solo ciertos elementos del HTML
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

# Carga los documentos filtrados desde la URL
docs = loader.load()

# Inicializa el splitter de texto para dividir documentos en partes más pequeñas
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Divide los documentos en partes según la configuración del splitter
splits = text_splitter.split_documents(docs)

# Crea un vector store a partir de los documentos divididos usando embeddings de OpenAI
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Crea un retriever a partir del vector store para realizar búsquedas
retriever = vectorstore.as_retriever()

# Obtiene un prompt predefinido desde el hub de LangChain
prompt = hub.pull("rlm/rag-prompt")

# Función para formatear documentos a una cadena
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define una cadena RAG (Retrieval-Augmented Generation) combinando retriever, formato y modelo
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

* Vea a continuación que el mensaje que hemos importado del hub tiene 2 variables: "contexto" y "pregunta".

In [ ]:
prompt

In [ ]:
rag_chain.invoke("What is Task Decomposition?")

#### Veamos en detalle la cadena LCEL:
* Como puedes ver, la primera parte de la cadena es un RunnableParallel (recuerda que RunnableParallel puede tener más de una sintaxis):

In [ ]:
rag_chain = (
    RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?")

Así es como funciona esta cadena cuando la invocamos:
* "¿Qué es la descomposición de tareas?" se pasa como entrada única.
* `context` ejecuta el recuperador sobre la entrada.
* format_docs ejecuta la función formateadora sobre la entrada.
* La entrada se asigna a `question`.
* el mensaje se define utilizando las variables `question` y `context` anteriores.
* el modelo se ejecuta con el mensaje anterior.
* el analizador de salida se ejecuta sobre la respuesta del modelo.

#### Nota: ¿qué hace la función de formateo anterior?
La función `format_docs` toma una lista de objetos llamados `docs`. Se espera que cada objeto de esta lista tenga un atributo llamado `page_content`, que almacena el contenido textual de cada documento.

El propósito de la función es extraer el `page_content` de cada documento en la lista `docs` y luego combinar estos contenidos en una sola cadena. El contenido de los diferentes documentos está separado por dos caracteres de nueva línea (`\n\n`), lo que significa que habrá una línea vacía entre el contenido de cada documento en la cadena final. Esta opción de formato hace que el contenido combinado sea más fácil de leer al separar claramente el contenido de los diferentes documentos.

A continuación, se muestra un desglose de cómo funciona la función:
1. La parte `for doc in docs` itera sobre cada objeto en la lista `docs`.
2. Para cada iteración, `doc.page_content` accede al atributo `page_content` del documento actual, que contiene su contenido textual.
3. El método `join` toma estos fragmentos de texto y los concatena en una sola cadena, insertando `\n\n` entre cada fragmento para garantizar que estén separados por una línea en blanco en el resultado final.

La función finalmente devuelve esta cadena única recién formateada que contiene todos los contenidos del documento, separados prolijamente por líneas en blanco.